Stephanie Beaver Capstone project convert scraped google reviews HTML to CSV with Author, date, text review, star review, reviewer experience and city, state, zip of the store the review was left on. May 31, 2025

Modified June 1 - June 3, 2025 because the first code didn't seem to work on a larger file. Modified to a code that work on larger file (and faster), and changed the way the location is captured from seperate columns city, state, and zip to full address to use as the unique identifier.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import re
import csv
from bs4 import BeautifulSoup, Comment
from collections import defaultdict
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
from tqdm import tqdm
import pandas as pd
import numpy as np


In [ ]:
# Set the date, tell it where its getting info from and where it is depositing
extraction_date = datetime(2025, 6, 5)
html_filename = '/content/drive/MyDrive/Colab Notebooks/Capstone/HTMLS/sportsmans_scraped.html'
csv_filename = '/content/drive/MyDrive/Colab Notebooks/Capstone/CSVs/sportsmans_converted.csv'

# How to convert the google reviews dates (2 days ago, 3 years ago etc)
def convert_relative_date(text, extraction_date):
    text = text.lower().strip()
    if text in ['today', 'just now']:
        return extraction_date.strftime('%b %Y')
    if text == 'yesterday':
        return (extraction_date - timedelta(days=1)).strftime('%b %Y')
    text = text.replace('a ', '1 ').replace('an ', '1 ')
    match = re.match(r'(\d+)\s+(hour|day|week|month|year)s?\s+ago', text)
    if not match:
        return extraction_date.strftime('%b %Y')
    quantity, unit = int(match.group(1)), match.group(2)
    delta = {
        'hour': timedelta(hours=quantity),
        'day': timedelta(days=quantity),
        'week': timedelta(weeks=quantity),
        'month': relativedelta(months=quantity),
        'year': relativedelta(years=quantity),
    }[unit]
    return (extraction_date - delta).strftime('%b %Y')

def extract_star_rating(review):
    span = review.find('span', attrs={'aria-label': re.compile(r'\d+(\.\d+)? star')})
    if span:
        match = re.search(r'(\d+)', span['aria-label'])
        return int(match.group(1)) if match else None
    return None

def extract_reviewer_experience(review):
    tag = review.find('span', class_='RfnDt') or review.find('div', class_='RfnDt')
    if tag:
        match = re.search(r'(\d+)\s+review', tag.get_text(strip=True).lower())
        return int(match.group(1)) if match else 0
    return 0

# === PARSE HTML ===
print("✅ Loading HTML file...")
with open(html_filename, 'r', encoding='utf-8') as f:
    soup = BeautifulSoup(f, 'lxml')

print("✅ Segmenting stores...")
store_segments = defaultdict(list)
current_address = "Unknown"

for el in tqdm(soup.descendants, desc="🔍 Scanning HTML", unit="elements"):
    if isinstance(el, Comment) and el.strip().startswith("Store Location:"):
        current_address = el.strip().replace("Store Location:", "").strip()
    elif getattr(el, 'name', None) == 'div' and 'jftiEf' in el.get('class', []):
        store_segments[current_address].append(el)

print(f"✅ Found {len(store_segments)} stores")

# === WRITE TO CSV ===
print("💾 Writing to CSV...")
with open(csv_filename, 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["Author", "Rating", "Date", "Review Text", "Reviewer Experience", "Store Address"])

    total_reviews = sum(len(r) for r in store_segments.values())
    progress = tqdm(total=total_reviews, desc="📝 Writing Reviews", unit="review")

    for address, reviews in store_segments.items():
        for review in reviews:
            try:
                author_tag = review.find("div", class_="d4r55")
                author = author_tag.get_text(strip=True) if author_tag else "Unknown"

                rating = extract_star_rating(review)

                date_tag = review.find("span", class_="rsqaWe")
                relative_date = date_tag.get_text(strip=True) if date_tag else "Unknown"
                date = convert_relative_date(relative_date, extraction_date)

                review_text_tag = review.find("span", class_="wiI7pd")
                review_text = review_text_tag.get_text(strip=True) if review_text_tag else ""

                experience = extract_reviewer_experience(review)

                writer.writerow([author, rating, date, review_text, experience, address])
                progress.update(1)
            except Exception as e:
                print(f"⚠️ Skipped review due to error: {e}")
                continue

    progress.close()

print(f"✅ Done! Converted {total_reviews} reviews into: {csv_filename}")

✅ Loading HTML file...
✅ Segmenting stores...


🔍 Scanning HTML: 2318983elements [00:01, 1190150.26elements/s]


✅ Found 47 stores
💾 Writing to CSV...


📝 Writing Reviews: 100%|██████████| 40603/40603 [00:21<00:00, 1908.63review/s]

✅ Done! Converted 40603 reviews into: /content/drive/MyDrive/Colab Notebooks/Capstone/CSVs/sportsmans_converted3.csv


In [ ]:
## I noticed that the scrape had picked up a few unrelated company reviews- so I dropped them
addresses_to_drop = [
    "129 N Main St, Logan, UT 84321",  #'The Sportsmans"
    "4535 SE 3rd St, Corvallis, OR 97333",  #Sportsmans storage"
    ""
]

deduplicated_df = deduplicated_df[~deduplicated_df['Store Address'].isin(addresses_to_drop)]

In [ ]:
print("Unique store addresses:", deduplicated_df['Store Address'].nunique())

Unique store addresses: 148


In [ ]:
## part of verifying the data that I had was complete
store_counts_by_state = deduplicated_df[deduplicated_df['Store Address'].str.lower() != 'unknown'] \
    .drop_duplicates(subset='Store Address') \
    ['Store Address'].str.extract(r', (\w{2}) ')[0] \
    .value_counts()

print(store_counts_by_state)

0
CA    17
WA    14
UT    14
AZ    10
CO     9
OR     8
PA     7
WY     7
ID     6
FL     6
AK     5
MI     4
VA     4
NV     4
SC     3
IN     3
MT     3
NM     3
NC     3
WI     2
NY     2
KY     2
WV     2
TN     2
IA     1
MS     1
ND     1
NE     1
MN     1
OH     1
LA     1
Name: count, dtype: int64


In [ ]:
Montana_addresses = deduplicated_df['Store Address'][
    deduplicated_df['Store Address'].str.contains(r', MT ', na=False)
].drop_duplicates()

print(Montana_addresses)

33272      2990 N Sanders St, Helena, MT 59601
74987    2323 N Reserve St, Missoula, MT 59808
64158      2214 Tschache Ln, Bozeman, MT 59718
Name: Store Address, dtype: object


In [ ]:
deduplicated_df.shape


(129183, 6)

In [ ]:
deduplicated_df.to_csv('/content/drive/MyDrive/Colab Notebooks/Capstone/CSVs/ALL_Sportsmans_Final.csv', index=False)

In [ ]:
## This code was added June 17 because further in the process my dates weren't working.. so I needed to split them and reformat them- then put them back into the csv for analysis

dates_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Capstone/CSVs/ALL_Sportsmans_Final.csv')
dates_df

,Author,Rating,Date,Review Text,Reviewer Experience,Store Address
0,Greg Kooker,5,2025-06-01,NaN,25,"921 SE Oralabor Rd, Ankeny, IA 50021"
1,Misty Maksin,5,2025-06-01,Ex ellent customer service,7,"1901 E Parks Hwy, Wasilla, AK 99654"
2,Mycool Williams,5,2025-06-01,It's an all-around great place to shop for all...,5,"520 Clock Tower Way, Crescent Springs, KY 41017"
3,Sam F,5,2025-06-01,NaN,210,"2231 E Rte 66, Flagstaff, AZ 86004"
4,Cyn Eaton,5,2025-06-01,NaN,4,"2231 E Rte 66, Flagstaff, AZ 86004"
...,...,...,...,...,...,...
129247,Karen Cline-Tardiff,5,Jun 2024,"Nice layout, good selection, not cluttered wit...",367,"436 Cornerstone Blvd, Hot Springs, AR 71913"
129248,ron,3,Jun 2024,"I know this location is relatively new, but I'...",150,"436 Cornerstone Blvd, Hot Springs, AR 71913"
129249,Dave Lindhag,4,Jun 2024,"Nice place, decent prices, gun selection is g...",237,"436 Cornerstone Blvd, Hot Springs, AR 71913"
129250,kc5vya,4,Jun 2024,NaN,117,"436 Cornerstone Blvd, Hot Springs, AR 71913"


In [ ]:
print(f"Initial rows: {len(dates_df)}")

Initial rows: 129252


In [ ]:
# Convert to string first so .str accessor works safely
date_strs = dates_df['Date'].astype(str)

# Identify ISO format rows (YYYY-MM-DD)
iso_mask = date_strs.str.match(r'^\d{4}-\d{2}-\d{2}$', na=False)

# Split into two DataFrames
df_iso = dates_df.loc[iso_mask].copy()
df_bmy = dates_df.loc[~iso_mask].copy()

# Parse ISO dates to datetime (this will infer the format)
df_iso['Date'] = pd.to_datetime(df_iso['Date'], errors='raise')

# Parse '%b %Y' dates explicitly to datetime (month-year format)
df_bmy['Date'] = pd.to_datetime(df_bmy['Date'], format='%b %Y', errors='raise')

# Normalize all dates to first day of the month (Period + to_timestamp)
df_iso['Date'] = df_iso['Date'].dt.to_period('M').dt.to_timestamp()
df_bmy['Date'] = df_bmy['Date'].dt.to_period('M').dt.to_timestamp()

# Recombine the two DataFrames (all dates are datetime and normalized)
dates_df = pd.concat([df_iso, df_bmy], ignore_index=True)

# Now dates_df['Date'] is all datetime dtype, normalized to first of month


In [ ]:
# Check the first few dates
print(dates_df['Date'].head(10))

# Check unique date formats by sampling a few entries
print(dates_df['Date'].drop_duplicates().sample(10, random_state=1).values)

# Verify that all dates match the '%b %Y' format (e.g., Jun 2025)
import re
pattern = r'^[A-Za-z]{3} \d{4}$'
non_matching = dates_df.loc[~dates_df['Date'].str.match(pattern), 'Date']
print(f"Non-matching dates count: {len(non_matching)}")
if len(non_matching) > 0:
    print(non_matching.unique())

# Value counts of dates to see distribution
print(dates_df['Date'].value_counts().sort_index())


0   2025-06-01
1   2025-06-01
2   2025-06-01
3   2025-06-01
4   2025-06-01
5   2025-06-01
6   2025-06-01
7   2025-06-01
8   2025-06-01
9   2025-06-01
Name: Date, dtype: datetime64[ns]
['2019-06-01T00:00:00.000000000' '2022-06-01T00:00:00.000000000'
 '2018-06-01T00:00:00.000000000' '2025-03-01T00:00:00.000000000'
 '2013-06-01T00:00:00.000000000' '2024-08-01T00:00:00.000000000'
 '2016-06-01T00:00:00.000000000' '2015-06-01T00:00:00.000000000'
 '2025-02-01T00:00:00.000000000' '2025-04-01T00:00:00.000000000']


AttributeError: Can only use .str accessor with string values!

In [ ]:
# Check which rows have null/missing values in 'Date' column
null_mask = dates_df['Date'].isnull()

# Count how many are null
num_nulls = null_mask.sum()
print(f"Number of nulls in 'Date': {num_nulls}")

# Optionally, display the rows where Date is null
null_rows = dates_df[null_mask]
print(null_rows)


Number of nulls in 'Date': 0
Empty DataFrame
Columns: [Author, Rating, Date, Review Text, Reviewer Experience, Store Address]
Index: []


In [ ]:
dates_df.shape


(129252, 6)

In [ ]:
dates_df.to_csv('/content/drive/MyDrive/Colab Notebooks/Capstone/CSVs/ALL_Sportsmans_Final.csv', index=False)

In [ ]:
dates_df.head()

,Author,Rating,Date,Review Text,Reviewer Experience,Store Address
0,Greg Kooker,5,Jun 2025,NaN,25,"921 SE Oralabor Rd, Ankeny, IA 50021"
1,Misty Maksin,5,Jun 2025,Ex ellent customer service,7,"1901 E Parks Hwy, Wasilla, AK 99654"
2,Mycool Williams,5,Jun 2025,It's an all-around great place to shop for all...,5,"520 Clock Tower Way, Crescent Springs, KY 41017"
3,Sam F,5,Jun 2025,NaN,210,"2231 E Rte 66, Flagstaff, AZ 86004"
4,Cyn Eaton,5,Jun 2025,NaN,4,"2231 E Rte 66, Flagstaff, AZ 86004"
